# EURO CUP2024 - GERMANY
## Predicting The European Champions

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/matches",True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/thirdplacecombination",True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/teamsgroups",True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/matchstats",True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/round_of_16",True)


Out[1]: True

## Table for Different Permutation and Combination
- There will be 15 different combination which will determine the knock out scenarios

In [0]:
%sql
CREATE or replace  TABLE ThirdPlaceCombination(
  ID INTEGER,
  SELECTEDGROUPS VARCHAR(4),
  Team1B VARCHAR(1),
  Team1C VARCHAR(1),
  Team1E VARCHAR(1),
  Team1F VARCHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO  ThirdPlaceCombination (ID, SELECTEDGROUPS, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

SELECT * FROM ThirdPlaceCombination;

ID,SELECTEDGROUPS,Team1B,Team1C,Team1E,Team1F
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


## Table for the Groups of Teams
- There will be 24 participating countries,which will be divided into 6 different groups.
- Each group will contain 4 different countries

In [0]:
%sql
CREATE or replace TABLE TeamsGroups (
    TeamID INT,
    TeamName VARCHAR(100),
    GroupName VARCHAR(1) 
);

INSERT INTO TeamsGroups (TeamID,TeamName,GroupName) VALUES
-- Group A
    (1, 'Germany','A'),
    (2, 'Scotland','A'),
    (3, 'Hungary','A'),
    (4, 'Switzerland','A'),

-- Group B
    (5, 'Spain','B'),
    (6, 'Croatia','B'),
    (7, 'Italy','B'),
    (8, 'Albania','B'),

-- Group C
    (9, 'Slovenia','C'),
    (10, 'Denmark','C'),
    (11, 'Serbia','C'),
    (12, 'England','C'),

-- Group D
    (13, 'Netherlands','D'),
    (14, 'Austria','D'),
    (15, 'France','D'),
    (16, 'Play-off Winner A','D'),

-- Group E
    (17, 'Belgium','E'),
    (18, 'Slovakia','E'),
    (19, 'Romania','E'),
    (20, 'Play-off Winner B','E'),

-- Group F
    (21, 'Turkey','F'),
    (22, 'Play-off Winner C','F'),
    (23, 'Portugal','F'),
    (24, 'Czech Republic','F');

SELECT * FROM TeamsGroups;

TeamID,TeamName,GroupName
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A
5,Spain,B
6,Croatia,B
7,Italy,B
8,Albania,B
9,Slovenia,C
10,Denmark,C


## Table for all the matches during euro cup
- This table will store all the matches of the euro cup.
- There will be total of 52 matches in the tournament across 10 different venues.
- **The Olympiastadion** in Berlin will host the tournament decider. 

In [0]:
%sql
CREATE or replace TABLE Matches (
    MatchID INT,
    Fixture varchar(50),
    GroupName varchar(1),
    HomeTeam varchar(20),
    AwayTeam varchar(20),
    HomeTeamGoals INT,
    AwayTeamGoals INT,
    MatchResult varchar(20),
    MatchStage varchar(20)
);

In [0]:
%sql
--Fixture of all the teams in group stage
INSERT INTO MATCHES
    SELECT
        ROW_NUMBER() over(order by 1) as MatchId,
        CONCAT(T1.TeamName, ' vs ', T2.TeamName) AS Fixture,
        T1.GroupName AS GroupName,
        T1.TeamName as HomeTeam,
        T2.TeamName as AwayTeam,
        CAST(rand() * 6 AS INT) AS GoalsScored, 
        CAST(rand() * 6 AS INT) AS GoalsAllowed,
        'None' as MatchResult,
        'Group Stage' as MatchStage
    FROM 
        TeamsGroups AS T1
    CROSS JOIN 
        TeamsGroups AS T2 where T1.GroupName = T2.GroupName AND T1.TeamID < T2.TeamID;


--Match result for the group stage
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 'Draw'
    END;

--All the matches from group stage
SELECT * FROM Matches;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
1,Germany vs Switzerland,A,Germany,Switzerland,5,1,Germany,Group Stage
2,Germany vs Hungary,A,Germany,Hungary,5,4,Germany,Group Stage
3,Germany vs Scotland,A,Germany,Scotland,0,5,Scotland,Group Stage
4,Scotland vs Switzerland,A,Scotland,Switzerland,0,1,Switzerland,Group Stage
5,Scotland vs Hungary,A,Scotland,Hungary,2,3,Hungary,Group Stage
6,Hungary vs Switzerland,A,Hungary,Switzerland,3,0,Hungary,Group Stage
7,Spain vs Albania,B,Spain,Albania,4,3,Spain,Group Stage
8,Spain vs Italy,B,Spain,Italy,3,0,Spain,Group Stage
9,Spain vs Croatia,B,Spain,Croatia,0,3,Croatia,Group Stage
10,Croatia vs Albania,B,Croatia,Albania,1,2,Albania,Group Stage


## Table to store Match Statistic during group stages
- This will store the satistic like W,L,D,GF,GA,GD,Points.
- All the Statistic wil be calculated using Aggregation Function.

In [0]:
%sql
--Match Statistics like win,loss,draw etc from the group stage matches
create or replace table MatchStats as 
    SELECT
        TeamID,
        GroupName,
         COUNT(*) AS MatchesPlayed,
        SUM(CASE 
            WHEN GoalsScored > GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Wins,
        SUM(CASE 
            WHEN GoalsScored = GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Draws,
        SUM(CASE 
            WHEN GoalsScored < GoalsAllowed THEN 1 
            ELSE 0 
        END) AS Losses,
        SUM(GoalsScored) AS GoalsScored,
        SUM(GoalsAllowed) AS GoalsAllowed,
        SUM(GoalsScored) - SUM(GoalsAllowed) AS GoalDifference,
        SUM(CASE
                WHEN GoalsScored > GoalsAllowed THEN 3  
                WHEN GoalsScored = GoalsAllowed THEN 1 
                ELSE 0                                
            END) AS Points
    FROM
        (SELECT
            GroupName,
            HomeTeam AS TeamID,
            HOMETEAMGOALS AS GoalsScored,
            AWAYTEAMGOALS AS GoalsAllowed
        FROM
            MATCHES
    
        UNION ALL
    
        SELECT
            GroupName,
            AwayTeam AS TeamID,
            AWAYTEAMGOALS AS GoalsScored,
            HOMETEAMGOALS AS GoalsAllowed
        FROM
            Matches) AS GroupMatches
    GROUP BY
        TeamID,groupname
        Order by groupname;

SELECT * FROM MatchStats;

TeamID,GroupName,MatchesPlayed,Wins,Draws,Losses,GoalsScored,GoalsAllowed,GoalDifference,Points
Germany,A,3,2,0,1,10,10,0,6
Scotland,A,3,1,0,2,7,4,3,3
Hungary,A,3,2,0,1,10,7,3,6
Switzerland,A,3,1,0,2,2,8,-6,3
Italy,B,3,2,0,1,10,11,-1,6
Croatia,B,3,1,0,2,8,7,1,3
Spain,B,3,2,0,1,7,6,1,6
Albania,B,3,1,0,2,9,10,-1,3
Serbia,C,3,1,1,1,8,5,3,4
Denmark,C,3,1,1,1,9,9,0,4


## Table to store all the qualifiying teams from each group
- This table will store the top 2 teams from each table.
- Next 4 teams wil be selected from third rank team from each group based on peformance.
- Points,GD,GA,W will be the peformance criteria.

In [0]:
%sql
-- Retrive top 2 teams from each groups
create or replace table round_of_16 as 
SELECT *
FROM (
    SELECT 
        TeamId,
        GroupName,
        points,
        goaldifference,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY points DESC, goaldifference DESC,goalsScored desc,wins desc) AS rnk 
    FROM 
        matchStats 
) AS Top2Teams
WHERE 
    rnk <= 2;

num_affected_rows,num_inserted_rows


In [0]:
%sql
--Best four teams from each group which are ranked third
insert into round_of_16
SELECT *
FROM (
    SELECT 
        TeamId,
        GroupName,
        points,
        goaldifference,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY points DESC, goaldifference desc,goalsScored desc,wins desc) AS rnk 
    FROM 
        matchStats 
) AS Top2Teams
WHERE 
    rnk = 3 LIMIT 4;

-- 16 teams for the knockout stages
SELECT * from round_of_16 order by GroupName;

TeamId,GroupName,points,goaldifference,rnk
Hungary,A,6,3,1
Scotland,A,3,3,3
Germany,A,6,0,2
Croatia,B,3,1,3
Spain,B,6,1,1
Italy,B,6,-1,2
England,C,4,0,3
Serbia,C,4,3,1
Denmark,C,4,0,2
Austria,D,3,-2,3


## Round of 16 matches 
- These matches will be played on the basis of the 15 different combination.
- We will be looking at the combination table to match our group combination.

- **Match 37:** Winner of Group A Vs. Runner-up Group C
- **Match 38:** Runner-up Group A Vs. Runner-up Group B
- **Match 39:** Winner of Group B Vs. 3rd Group A/D/E/F  	
- **Match 40:** Winner of Group C Vs. 3rd Group D/E/F 
- **Match 41:** Winner of Group F vs. 3rd Group A/B/C
- **Match 42:** Runner-up Group D vs. Runner-up Group E
- **Match 43:** Winner of Group E vs. 3rd Group A/B/C/D
- **Match 44:** Winner of Group D vs. Runner-up Group F


In [0]:
%sql
INSERT INTO matches (MatchId, fixture, GroupName,HomeTeam,AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
WITH Combination AS (
    SELECT CONCAT_WS('', sort_array(Collect_list(GroupName))) AS GroupCombination
    FROM round_of_16
    WHERE rnk = 3 
),
keys AS (
    SELECT * FROM thirdplacecombination
    WHERE selectedgroups = (SELECT GroupCombination FROM Combination)
)
SELECT 
    37 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'A' AND r1.rnk = 1 AND r2.groupname = 'C' AND r2.rnk = 2
    

UNION ALL

SELECT 
    38 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'A' AND r1.rnk = 2 AND r2.groupname = 'B' AND r2.rnk = 2


UNION ALL

SELECT 
    39 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'B' AND r1.rnk = 1 AND r2.groupname = (SELECT Team1B FROM keys) AND r2.rnk = 3
  

UNION ALL

SELECT 
    40 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
   round_of_16 r2 ON r1.groupname = 'C' AND r1.rnk = 1 AND r2.groupname = (select Team1C from keys) AND r2.rnk = 3
    

UNION ALL

SELECT 
    41 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'F' AND r1.rnk = 1 AND r2.groupname = (select Team1F from keys) AND r2.rnk = 3
    

UNION ALL

SELECT 
    42 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'D' AND r1.rnk = 2 AND r2.groupname = 'E' AND r2.rnk = 2

UNION ALL

SELECT 
    43 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
  round_of_16 r2 ON r1.groupname = 'E' AND r1.rnk = 1 AND r2.groupname = (select Team1E from keys) AND r2.rnk = 3

UNION ALL

SELECT 
    44 AS MatchId,
    CONCAT(r1.teamid, ' Vs. ', r2.teamid) AS fixture,
    'N' AS GroupName,
    r1.TEAMID AS HomeTeam,
    r2.TEAMID AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'R16' AS MatchStage
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.groupname = 'D' AND r1.rnk = 2 AND r2.groupname = 'F' AND r2.rnk = 2


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
-- UPDATE THE MATCH RESULT FOR THE R16
UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
         ELSE 
            CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
  WHERE MatchStage = "R16";


num_affected_rows
8


## Matches for the Quater-Final Round 
- **Match 45:** Winner Match 39 vs. Winner Match 37
- **Match 46:** Winner Match 41 vs. Winner Match 42
- **Match 47:** Winner Match 43 vs. Winner match 44
- **Match 48:** Winner Match 40 vs. Winner Match 38

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    45 AS MatchId,
    CONCAT(QF1.MATCHRESULT, ' Vs. ', QF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    QF1.MATCHRESULT AS HomeTeam,
    QF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Quarter-Final' AS MatchStage
FROM 
    matches QF1
JOIN 
    matches QF2 ON 
        QF1.MATCHID = 39 AND QF2.MatchId = 37

UNION ALL

SELECT 
    46 AS MatchId,
    CONCAT(QF1.MATCHRESULT, ' Vs. ', QF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    QF1.MATCHRESULT AS HomeTeam,
    QF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Quarter-Final' AS MatchStage
FROM 
    matches QF1
JOIN 
    matches QF2 ON 
        QF1.MATCHID = 41 AND QF2.MatchId = 42

UNION ALL

SELECT 
    47 AS MatchId,
    CONCAT(QF1.MATCHRESULT, ' Vs. ', QF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    QF1.MATCHRESULT AS HomeTeam,
    QF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Quarter-Final' AS MatchStage
FROM 
    matches QF1
JOIN 
    matches QF2 ON 
        QF1.MATCHID = 43 AND QF2.MatchId = 44

UNION ALL

SELECT 
    48 AS MatchId,
    CONCAT(QF1.MATCHRESULT, ' Vs. ', QF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    QF1.MATCHRESULT AS HomeTeam,
    QF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Quarter-Final' AS MatchStage
FROM 
    matches QF1
JOIN 
    matches QF2 ON 
        QF1.MATCHID = 40 AND QF2.MatchId = 38;

UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
        CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
WHERE MatchStage = 'Quarter-Final';

num_affected_rows
4


In [0]:
%sql 
select * from matches order by MatchID;
-- DELETE FROM matches
-- WHERE matchstage IN ('R16', 'Quarter-Final');

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
1,Germany vs Switzerland,A,Germany,Switzerland,5,1,Germany,Group Stage
2,Germany vs Hungary,A,Germany,Hungary,5,4,Germany,Group Stage
3,Germany vs Scotland,A,Germany,Scotland,0,5,Scotland,Group Stage
4,Scotland vs Switzerland,A,Scotland,Switzerland,0,1,Switzerland,Group Stage
5,Scotland vs Hungary,A,Scotland,Hungary,2,3,Hungary,Group Stage
6,Hungary vs Switzerland,A,Hungary,Switzerland,3,0,Hungary,Group Stage
7,Spain vs Albania,B,Spain,Albania,4,3,Spain,Group Stage
8,Spain vs Italy,B,Spain,Italy,3,0,Spain,Group Stage
9,Spain vs Croatia,B,Spain,Croatia,0,3,Croatia,Group Stage
10,Croatia vs Albania,B,Croatia,Albania,1,2,Albania,Group Stage


## Matches For the Semi-Final Round

- **Match 49:** Winner Match 45 vs. Winner match 46
- **Match 50:** Winner Match 47 vs. Winner Match 48

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 48 AS MatchId,
    CONCAT(SF1.MATCHRESULT, ' Vs. ', SF2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    SF1.MATCHRESULT AS HomeTeam,
    SF2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Semi-Final' AS MatchStage
FROM 
    matches SF1
JOIN 
    matches SF2 ON (
        (SF1.MATCHID = 45 AND SF2.MatchId = 46) OR
        (SF1.MATCHID = 47 AND SF2.MatchId = 48)
    );

UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                  WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                  ELSE AwayTeam
              END
    END
WHERE MatchStage = 'Semi-Final';

num_affected_rows
2


## Match For Third Place

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
WITH LosingTeams AS (
    SELECT
        CASE
            WHEN MATCHRESULT != HOMETEAM THEN HOMETEAM
            ELSE AWAYTEAM
        END AS LosingTeam
    FROM
        matches
    WHERE 
        MATCHSTAGE = 'Semi-Final'
)

SELECT 
    Row_Number() OVER(ORDER BY 1) + 50 AS MatchId,
    CONCAT(T1.LosingTeam, ' Vs. ', T2.LosingTeam) AS fixture,
    'N' as GroupName,
    T1.LosingTeam AS HomeTeam,
    T2.LosingTeam AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Third-Place' AS MatchStage
FROM 
    LosingTeams T1
JOIN 
    LosingTeams T2 ON T1.LosingTeam < T2.LosingTeam;

UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                  WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                  ELSE AwayTeam
              END
    END
WHERE MatchStage = 'Third-Place';


num_affected_rows
1


## Match For the Final

- **Match 52** Winner Match 49 vs. Winner Match 50

In [0]:
%sql
INSERT INTO matches (MatchId, fixture,GroupName, HomeTeam, AwayTeam, HomeTeamGoals, AwayTeamGoals, MatchResult, MatchStage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 51 AS MatchId,
    CONCAT(F1.MATCHRESULT, ' Vs. ', F2.MATCHRESULT) AS fixture,
    'N' as GroupName,
    F1.MATCHRESULT AS HomeTeam,
    F2.MATCHRESULT AS AwayTeam,
    CAST(rand() * 6 AS INT) AS GoalsScored, 
    CAST(rand() * 6 AS INT) AS GoalsAllowed,
    'None' AS MatchResult,
    'Final' AS MatchStage
FROM 
    matches F1
JOIN 
    matches F2 ON 
        F1.MATCHID = 49 AND F2.MatchId = 50;

UPDATE Matches
SET MatchResult = 
    CASE
        WHEN HOMETEAMGOALS > AWAYTEAMGOALS THEN HomeTeam
        WHEN HOMETEAMGOALS < AWAYTEAMGOALS THEN AwayTeam
        ELSE 
          CASE 
                WHEN (HomeTeamGoals + 1) % 2 = 1 THEN HomeTeam
                ELSE AwayTeam
            END
    END
WHERE MatchStage = 'Final';

num_affected_rows
1


In [0]:
%sql SELECT * FROM matches ORDER BY MatchID;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
1,Germany vs Switzerland,A,Germany,Switzerland,5,1,Germany,Group Stage
2,Germany vs Hungary,A,Germany,Hungary,5,4,Germany,Group Stage
3,Germany vs Scotland,A,Germany,Scotland,0,5,Scotland,Group Stage
4,Scotland vs Switzerland,A,Scotland,Switzerland,0,1,Switzerland,Group Stage
5,Scotland vs Hungary,A,Scotland,Hungary,2,3,Hungary,Group Stage
6,Hungary vs Switzerland,A,Hungary,Switzerland,3,0,Hungary,Group Stage
7,Spain vs Albania,B,Spain,Albania,4,3,Spain,Group Stage
8,Spain vs Italy,B,Spain,Italy,3,0,Spain,Group Stage
9,Spain vs Croatia,B,Spain,Croatia,0,3,Croatia,Group Stage
10,Croatia vs Albania,B,Croatia,Albania,1,2,Albania,Group Stage


In [0]:
%sql
select * from matches where MatchStage = 'R16' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
37,Hungary Vs. Denmark,N,Hungary,Denmark,3,3,Denmark,R16
38,Germany Vs. Italy,N,Germany,Italy,4,2,Germany,R16
39,Spain Vs. Scotland,N,Spain,Scotland,0,4,Scotland,R16
40,Serbia Vs. Austria,N,Serbia,Austria,5,3,Serbia,R16
41,Turkey Vs. England,N,Turkey,England,3,0,Turkey,R16
42,France Vs. Belgium,N,France,Belgium,1,5,Belgium,R16
43,Slovakia Vs. Croatia,N,Slovakia,Croatia,5,4,Slovakia,R16
44,France Vs. Portugal,N,France,Portugal,2,3,Portugal,R16


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Quarter-Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
45,Scotland Vs. Denmark,N,Scotland,Denmark,5,3,Scotland,Quarter-Final
46,Turkey Vs. Belgium,N,Turkey,Belgium,5,1,Turkey,Quarter-Final
47,Slovakia Vs. Portugal,N,Slovakia,Portugal,2,0,Slovakia,Quarter-Final
48,Serbia Vs. Germany,N,Serbia,Germany,4,3,Serbia,Quarter-Final


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Semi-Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
49,Slovakia Vs. Serbia,N,Slovakia,Serbia,2,0,Slovakia,Semi-Final
50,Scotland Vs. Turkey,N,Scotland,Turkey,4,2,Scotland,Semi-Final


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Third-Place' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
51,Serbia Vs. Turkey,N,Serbia,Turkey,1,4,Turkey,Third-Place


In [0]:
%sql
SELECT * FROM Matches where MatchStage = 'Final' order by MatchId;

MatchID,Fixture,GroupName,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,MatchResult,MatchStage
52,Slovakia Vs. Scotland,N,Slovakia,Scotland,1,2,Scotland,Final


# Euro Cup 2024 Visualization

In [0]:
#import libraries
import pandas as pd
from pyspark.sql import SparkSession


In [0]:
#Spark Session
spark = SparkSession.builder \
    .appName("euro_cup_visulization") \
    .getOrCreate()

## Group Stage Table Visualization

In [0]:
#Query from spark sql for different stages
all_teams_stats = spark.sql(""" SELECT *
FROM (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY GroupName ORDER BY points DESC, goaldifference desc,goalsScored desc,wins desc) AS rnk 
    FROM 
        matchStats 
) AS Top2Teams""")
# round_of_16 = spark.sql("SELECT HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals FROM Matches WHERE MatchStage = 'R16' ORDER BY MatchId")
quater_final = spark.sql("SELECT HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals FROM Matches WHERE MatchStage = 'Quarter-Final' ORDER BY MatchId")
semi_final = spark.sql("SELECT HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals  FROM Matches WHERE MatchStage = 'Semi-Final' ORDER BY MatchId desc")
final = spark.sql("SELECT HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals  FROM Matches WHERE MatchStage = 'Final' ORDER BY MatchId")
third_place = spark.sql("SELECT MatchId  FROM Matches WHERE MatchStage = 'Third-Place' ORDER BY MatchId")


In [0]:
round_of_16 = spark.sql("""
    SELECT
        CASE WHEN matchid IN (39, 37, 41, 42) THEN 'Left' ELSE 'Right' END AS Side,
        MatchID,
        HomeTeam,
        AwayTeam,
        HomeTeamGoals,
        AwayTeamGoals
    FROM
        matches m
    WHERE
        m.matchid IN (39, 37, 41, 42, 43, 44, 40, 38) 
        AND MatchStage = 'R16'
    ORDER BY
        CASE 
            WHEN matchid = 39 THEN 1
            WHEN matchid = 37 THEN 2
            WHEN matchid = 41 THEN 3
            WHEN matchid = 42 THEN 4
            WHEN matchid = 43 THEN 5
            WHEN matchid = 44 THEN 6
            WHEN matchid = 40 THEN 7
            WHEN matchid = 38 THEN 8
        END
""")
round_of_16_df = round_of_16.toPandas()
display(round_of_16)

# new_r16_df = new_r16.toPandas()
# display(new_r16_df)


Side,MatchID,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals
Left,39,Spain,Scotland,0,4
Left,37,Hungary,Denmark,3,3
Left,41,Turkey,England,3,0
Left,42,France,Belgium,1,5
Right,43,Slovakia,Croatia,5,4
Right,44,France,Portugal,2,3
Right,40,Serbia,Austria,5,3
Right,38,Germany,Italy,4,2


In [0]:
#Convert Spark DataFrame to Pandas Dataframe
all_teams_stats_df = all_teams_stats.toPandas()
# round_of_16_df = round_of_16.toPandas()
quater_final_df = quater_final.toPandas()
semi_final_df = semi_final.toPandas()
final_df = final.toPandas()
third_place_df = third_place.toPandas()

display(all_teams_stats_df)

TeamID,GroupName,MatchesPlayed,Wins,Draws,Losses,GoalsScored,GoalsAllowed,GoalDifference,Points,rnk
Hungary,A,3,2,0,1,10,7,3,6,1
Germany,A,3,2,0,1,10,10,0,6,2
Scotland,A,3,1,0,2,7,4,3,3,3
Switzerland,A,3,1,0,2,2,8,-6,3,4
Spain,B,3,2,0,1,7,6,1,6,1
Italy,B,3,2,0,1,10,11,-1,6,2
Croatia,B,3,1,0,2,8,7,1,3,3
Albania,B,3,1,0,2,9,10,-1,3,4
Serbia,C,3,1,1,1,8,5,3,4,1
Denmark,C,3,1,1,1,9,9,0,4,2


In [0]:
# Create table visualization DataFrame
table_visualization = pd.DataFrame({
    'Pos': all_teams_stats_df['rnk'],
    'Teams': all_teams_stats_df['TeamID'],
    'Group': all_teams_stats_df['GroupName'],
    'Pld': all_teams_stats_df['MatchesPlayed'],
    'W': all_teams_stats_df['Wins'],
    'D': all_teams_stats_df['Draws'],
    'L': all_teams_stats_df['Losses'],
    'GF': all_teams_stats_df['GoalsScored'],
    'GA': all_teams_stats_df['GoalsAllowed'],
    'GD': all_teams_stats_df['GoalDifference'],
    'Pts': all_teams_stats_df['Points']
})

blank_row = {col: '' for col in table_visualization.columns}


blank_row_df = pd.DataFrame([blank_row])

# Divide the each teams into group of four
groups = [table_visualization.iloc[i:i+4] for i in range(0, len(table_visualization), 4)]

for i in range(len(groups)):
    if i < len(groups) - 1:
        groups[i] = pd.concat([groups[i], blank_row_df], ignore_index=True)

visualization_with_space = pd.concat(groups).reset_index(drop=True)

# Calculate the middle part of the full table
half_table_index = len(visualization_with_space) // 2
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

display(visualization_with_space)
print(half_table_index)


Pos,Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
1,Hungary,A,3,2,0,1,10,7,3,6
2,Germany,A,3,2,0,1,10,10,0,6
3,Scotland,A,3,1,0,2,7,4,3,3
4,Switzerland,A,3,1,0,2,2,8,-6,3
,,,,,,,,,,
1,Spain,B,3,2,0,1,7,6,1,6
2,Italy,B,3,2,0,1,10,11,-1,6
3,Croatia,B,3,1,0,2,8,7,1,3
4,Albania,B,3,1,0,2,9,10,-1,3
,,,,,,,,,,


14


In [0]:
#Table information for the Group A,B,C
left_table = pd.DataFrame(columns=['Pos','Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
left_table = pd.concat([left_table, pd.DataFrame([blank_row])], ignore_index=True)
left_table = pd.concat([left_table, visualization_with_space.iloc[:half_table_index]]).reset_index(drop=True)

#Table information for the group C,D,E
right_table = pd.DataFrame(columns=['Pos','Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
right_table = pd.concat([right_table, visualization_with_space.iloc[half_table_index +1:]]).reset_index(drop=True)
right_table = pd.concat([pd.DataFrame([blank_row]), right_table], ignore_index=True)

display(left_table)
display(right_table)

Pos,Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,,
1,Hungary,A,3,2,0,1,10,7,3,6
2,Germany,A,3,2,0,1,10,10,0,6
3,Scotland,A,3,1,0,2,7,4,3,3
4,Switzerland,A,3,1,0,2,2,8,-6,3
,,,,,,,,,,
1,Spain,B,3,2,0,1,7,6,1,6
2,Italy,B,3,2,0,1,10,11,-1,6
3,Croatia,B,3,1,0,2,8,7,1,3
4,Albania,B,3,1,0,2,9,10,-1,3


Pos,Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,,
1,Play-off Winner A,D,3,3,0,0,7,3,4,9
2,France,D,3,2,0,1,8,4,4,6
3,Austria,D,3,1,0,2,8,10,-2,3
4,Netherlands,D,3,0,0,3,0,6,-6,0
,,,,,,,,,,
1,Slovakia,E,3,2,1,0,10,4,6,7
2,Belgium,E,3,2,0,1,8,10,-2,6
3,Romania,E,3,1,1,1,4,4,0,4
4,Play-off Winner B,E,3,0,0,3,6,10,-4,0


## Visulizing the Round of 16 Qualification By Splitting into two Pools Left and Right

In [0]:
# Create an empty DataFrame for round of sixteen
visualize_r16 = pd.DataFrame(columns=['Round_of_16', 'rest'])

for _ in range(2):
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

for index, row in round_of_16_df.iterrows():
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['HomeTeam']} ({row['HomeTeamGoals']})"]})], ignore_index=True)
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['AwayTeam']} ({row['AwayTeamGoals']})"]})], ignore_index=True)

    for _ in range(2):
        visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

visualize_r16['rest'] = ''

splited_r16_index = len(visualize_r16) // 2
print(splited_r16_index)
display(visualize_r16)


17


Round_of_16,rest
,
,
Spain (0),
Scotland (4),
,
,
Hungary (3),
Denmark (3),
,
,


In [0]:
left_side_r16 = visualize_r16.iloc[:splited_r16_index + 1].reset_index(drop = True)
right_side_r16 = visualize_r16.iloc[splited_r16_index - 1:].reset_index(drop = True)
display(left_side_r16)
display(right_side_r16)

Round_of_16,rest
,
,
Spain (0),
Scotland (4),
,
,
Hungary (3),
Denmark (3),
,
,


Round_of_16,rest
,
,
Slovakia (5),
Croatia (4),
,
,
France (2),
Portugal (3),
,
,


## Visulizing the Quater-Final Qualification

In [0]:
# Dataframe for quater final
visualize_qf = pd.DataFrame(columns=['Quater_Final'])

for _ in range(4):
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)

for index, row in quater_final_df.iterrows():
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['HomeTeam']} ({row['HomeTeamGoals']})"]})], ignore_index=True)
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['AwayTeam']} ({row['AwayTeamGoals']})"]})], ignore_index=True)

    for _ in range(6):
        visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)
visualize_qf['rest'] = ''
splited_qf_index = len(visualize_qf) // 2
display(visualize_qf)


Quater_Final,rest
,
,
,
,
Scotland (5),
Denmark (3),
,
,
,
,


In [0]:
left_side_qf = pd.DataFrame(columns=['Quater_Final'])
left_side_qf = pd.concat([left_side_qf,visualize_qf.iloc[:splited_qf_index]]).reset_index(drop=True)
right_side_qf = pd.DataFrame(columns=['Quater_Final'])
right_side_qf = pd.concat([right_side_qf,visualize_qf.iloc[splited_qf_index-2:]]).reset_index(drop=True)


display(left_side_qf)
display(right_side_qf)

Quater_Final,rest
,
,
,
,
Scotland (5),
Denmark (3),
,
,
,
,


Quater_Final,rest
,
,
,
,
Slovakia (2),
Portugal (0),
,
,
,
,


## Visulizing the Semi-Final Qualification

In [0]:

visualize_sf = pd.DataFrame(columns=['Semi_Final'])

for _ in range(8):
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)
    
for index, row in semi_final_df.iterrows():
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['HomeTeam']} ({row['HomeTeamGoals']})"]})], ignore_index=True)
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['AwayTeam']} ({row['AwayTeamGoals']})"]})], ignore_index=True)
    for _ in range(6):
        visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)
visualize_sf['rest'] = ''
splited_sf_index = len(visualize_sf) // 2
print(splited_sf_index)
display(visualize_sf)


12


Semi_Final,rest
,
,
,
,
,
,
,
,
Scotland (4),
Turkey (2),


In [0]:
left_side_sf = pd.DataFrame(columns=['Semi_Final'])
left_side_sf = pd.concat([left_side_sf, visualize_sf.iloc[:splited_sf_index]]).reset_index(drop=True)

for _ in range(5):
    left_side_sf = pd.concat([left_side_sf, pd.DataFrame({'Semi_Final': [''], 'rest': ['']})], ignore_index=True)

right_side_sf = pd.DataFrame({'Semi_Final': [''] * 7, 'rest': [''] * 7})
right_side_sf = pd.concat([right_side_sf, visualize_sf.iloc[splited_sf_index+3:]]).reset_index(drop=True)
display(left_side_sf)
display(right_side_sf)

Semi_Final,rest
,
,
,
,
,
,
,
,
Scotland (4),
Turkey (2),


Semi_Final,rest
,
,
,
,
,
,
,
,
Slovakia (2),
Serbia (0),


## Visulizing the Final Qualification

In [0]:
visualize_final = pd.DataFrame(columns=['Final'])


for _ in range(8):
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)


for index, row in final_df.iterrows():
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['HomeTeam']} ({row['HomeTeamGoals']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['AwayTeam']} ({row['AwayTeamGoals']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['EUROPEAN CHAMPION']})], ignore_index=True)

visualize_final['rest'] = ''
display(visualize_final)


Final,rest
,
,
,
,
,
,
,
,
Slovakia (1),
Scotland (2),


In [0]:
left_side_qf.columns = [f"{col}q2" for col in left_side_qf.columns]

merged_df = pd.concat([left_side_r16, left_side_qf], axis=1)

left_side_sf.columns = [f"{col}s1" for col in left_side_sf.columns]
merged_df = pd.concat([merged_df, left_side_sf], axis=1)

visualize_final.columns = [f"{col}f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

right_side_sf.columns = [f"{col}s2" for col in right_side_sf.columns]

merged_df = pd.concat([merged_df, right_side_sf], axis=1)

right_side_qf.columns = [f"{col}q1" for col in right_side_qf.columns]
# Concatenate the DataFrames
merged_df = pd.concat([merged_df, right_side_qf], axis=1)

right_side_r16.columns = [f"{col}r2" for col in right_side_r16.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df,right_side_r16], axis=1)

left_table.columns = [f"{col}g1" for col in left_table.columns]
right_table.columns = [f"{col}g2" for col in right_table.columns]

# Concatenate the DataFrames
merged_df = pd.concat([left_table,merged_df, right_table], axis=1)

merged_df.fillna('', inplace=True)


display(merged_df)

Posg1,Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2,restq2,Semi_Finals1,rests1,Finalf,restf,Semi_Finals2,rests2,Quater_Finalq1,restq1,Round_of_16r2,restr2,Posg2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Hungary,A,3,2,0,1,10,7,3,6,,,,,,,,,,,,,,,1,Play-off Winner A,D,3,3,0,0,7,3,4,9
2,Germany,A,3,2,0,1,10,10,0,6,Spain (0),,,,,,,,,,,,Slovakia (5),,2,France,D,3,2,0,1,8,4,4,6
3,Scotland,A,3,1,0,2,7,4,3,3,Scotland (4),,,,,,,,,,,,Croatia (4),,3,Austria,D,3,1,0,2,8,10,-2,3
4,Switzerland,A,3,1,0,2,2,8,-6,3,,,Scotland (5),,,,,,,,Slovakia (2),,,,4,Netherlands,D,3,0,0,3,0,6,-6,0
,,,,,,,,,,,,,Denmark (3),,,,,,,,Portugal (0),,,,,,,,,,,,,,
1,Spain,B,3,2,0,1,7,6,1,6,Hungary (3),,,,,,,,,,,,France (2),,1,Slovakia,E,3,2,1,0,10,4,6,7
2,Italy,B,3,2,0,1,10,11,-1,6,Denmark (3),,,,,,,,,,,,Portugal (3),,2,Belgium,E,3,2,0,1,8,10,-2,6
3,Croatia,B,3,1,0,2,8,7,1,3,,,,,Scotland (4),,Slovakia (1),,Slovakia (2),,,,,,3,Romania,E,3,1,1,1,4,4,0,4
4,Albania,B,3,1,0,2,9,10,-1,3,,,,,Turkey (2),,Scotland (2),,Serbia (0),,,,,,4,Play-off Winner B,E,3,0,0,3,6,10,-4,0


In [0]:

# # Concatenate the DataFrames
# merged_df = pd.concat([left_table, left_side_r16, left_side_qf, left_side_sf, visualize_final, right_side_sf, right_side_qf, right_side_r16, right_table], axis=1)
# merged_df.fillna('', inplace=True)
# # Display the DataFrame
# display(merged_df)
